# Assignment 1
Remote Sensing & Big Data
Assignment 1
In this assignment, you will implement the map-reduce principles and some basic image operations (e.g. morphological operations) on the Landsat 7 image collection in the Google Earth Engine (GEE). Below you find the different steps to perform and the questions that come with these steps.

When handing in the assignment on Brightspace, we ask you to upload the following:

- Your GEE scripts:
- When working in the javascript/web-interface: please add your code as appendix to your report (e.g. printed as pdf) and a hyperlink to the working code on GEE.
- When working in the python-interface: please add your jupyter-notebook and a printed version (e.g. as pdf) of this notebook.
- A motivation for each step in your code. This can be done as comments in your code or separate in the report.
- Response to the questions, which are highlighted in red.

If you do not provide your code and/or motivation for each step, we will not grade the assignment resulting in a fail score.


In [9]:
import ee
import geemap
from geemap import cartoee
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [10]:
def setup():
    ee.Authenticate()
    ee.Initialize()
#setup()

In [74]:
datedict = ls.reduceColumns(ee.Reducer.minMax(),['DATE_ACQUIRED']).getInfo()
enddate,startdate = datedict.values()
endyear = ee.Date(enddate).format('YYYY').getInfo()
startyear = ee.Date(startdate).format('YYYY').getInfo()

In [79]:
def annual_comp(collection, year):
    st = f'{year}-01-01'
    end = f'{year}-12-31'
    return collection.filterDate(st,end)

def all_annual_comps(collection: ee.ImageCollection):
    datedict = collection.reduceColumns(ee.Reducer.minMax(),['DATE_ACQUIRED']).getInfo()
    enddate,startdate = datedict.values()
    endyear = int(ee.Date(enddate).format('YYYY').getInfo())
    startyear = int(ee.Date(startdate).format('YYYY').getInfo())
    
    annual_collection_list = []
    
    for year in range(startyear,endyear):
        yearcollection = annual_comp(collection,year)
        annual_collection_list.append(yearcollection)
    
    return annual_collection_list
        

In [11]:
def map_image(image,name):
    projection = ccrs.epsg(26985)
    vp =  {'bands':[f'B3_{name}',f'B2_{name}',f'B1_{name}'],
              'min':0,
             'max':0.4,
            'gamma': 1.2,
              }
        
    fig = plt.figure(figsize=(10,7))

    # use cartoee to get a map
    ax = cartoee.get_map(image,region=bbox_coord,vis_params=vp,proj=projection)

    # add gridlines to the map at a specified interval
    #cartoee.add_gridlines(ax,interval=[60,30],linestyle=":")

    # add coastlines using the cartopy api
    #ax.coastlines(color="red")

    plt.show()

In [12]:
def make_reduced_images(imcollection: ee.ImageCollection,Icname):
    """
    Accepts an image collection, applies the four reducers

    returns dictionary of reducer:image with
    """
    # set up empty dict
    reduced_images = {}

    reducers = {
    'mean':ee.Reducer.mean(),
    'median':ee.Reducer.median(),
    'p5':ee.Reducer.percentile(percentiles=[5]),
    'p95':ee.Reducer.percentile(percentiles=[95])           
    }

    for x in reducers:
        name = x
        reducer = reducers[x]
        reduced_image = imcollection.reduce(reducer)
        assert type(name) == str  # check that 
        assert type(reduced_image) == ee.Image
        reduced_images.update({name:reduced_image})

    return reduced_images

In [13]:
def map_reduced_images(imcollection: ee.ImageCollection):
    reduced_images = make_reduced_images(imcollection,'_')
    for reduced_image_name in reduced_images:
        print(f'Mapping {reduced_image_name}')
        reduced_image = reduced_images[reduced_image_name]
        map_image(reduced_image,reduced_image_name)
        

In [14]:
def add_composites_to_map(imcollection: ee.ImageCollection,m: geemap.Map,Icname):
    """
    Accepts an image collection, a map, and a name for this image colection
    
    returns the map with mean, median, count, 0.05 cut and 0.95 percentile cut of the image collection
    """
    # add count first because we 
    m.add_ee_layer(imcollection.reduce(ee.Reducer.count()),name='count')
    
    images = make_reduced_images(imcollection,Icname)
    
    for imagename in images:
        name = imagename
        reduced_image = images[imagename]
        vp =  {'bands':[f'B3_{name}',f'B2_{name}',f'B1_{name}'],
              'min':0,
             'max':0.4,
            'gamma': 1.2,
              }
        
        m.add_ee_layer(reduced_image,vis_params=vp,name=Icname+'-'+name)
        
    return m
    

## Step 1 Data loading
- Load the USGS Landsat 7 Top of Atmosphere Reflectance Tier 1 data over a region of your interest. FYI: if you want to know what the difference between the different Tiers is: https://www.usgs.gov/media/videos/landsat-collections-what-are-tiers

In [15]:
house_coords = [38, -76.21]  # lat long
box_size= 0.00  # in degrees
bbox_coord = [house_coords[1]-box_size,house_coords[0]-box_size,house_coords[1]+box_size,house_coords[0]+box_size]
poi = ee.Geometry.Point(house_coords[::-1])  # ee geometry needs long, lat so list is reversed
roi = ee.Geometry.BBox(*bbox_coord)

Map = geemap.Map(center = house_coords,zoom=8)
Map.addLayerControl()

ls = ee.ImageCollection("LANDSAT/LE07/C01/T1_TOA").filterBounds(poi)

## Step 2 Naive annual composites
- Apply the map-reduce principle to convert the image collection of Step 1 into annual composites with different filtering & reducing conditions. Within this step, implement:
    - A: Different filtering conditions based cloud cover metadata:
        - a1: no filtering

In [16]:
a1 = ls # a1 is just the landsat with no filtering
Map = add_composites_to_map(a1,Map,'a1')

        - a2: filtering on <50% cloud cover 

In [17]:
a2 = ls.filterMetadata('CLOUD_COVER','less_than',50)
Map = add_composites_to_map(a2,Map,'a2')

        - a3: filtering in <20% cloud cover

In [18]:
a3 = ls.filterMetadata('CLOUD_COVER','less_than',20)
Map = add_composites_to_map(a3,Map,'ac3')

In [19]:
Map

Map(center=[38, -76.21], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

- B: Use the result of a2 and test the effect of masking the clouds & cloud-shadows using the BQA bitmask: 
    - b0: without cloud masking 

In [116]:
Map = geemap.Map(center = house_coords,zoom=10)

In [117]:
b0 = ls.filterMetadata('CLOUD_COVER','less_than',50)

Map = add_composites_to_map(b0,Map,'No Masking')

    - b1: with the clouds masked 

In [181]:
def cld_mask(image: ee.Image):
    cloudbit = 1 << 4
    bqa = image.select('BQA')
    cloudmask = bqa.bitwise_and(cloudbit).eq(0)
    cloudmask = cloudmask.rename(['cloudMask'])
    image = image.addBands(cloudmask)
    image.updateMask('cloudMask')
    return image

In [190]:
def cld_shdw_mask(image):
    """
    takes a landsat 7 image
    
    returns the same image with a band called cloudMask based on where the there is a shadow OR a cloud
    """
    cloudbit = 1 << 4
    cloudshadowbit = 2 << 7
    bqa = image.select('BQA')
    #cloudmask = bqa.bitwise_and(cloudbit).Or(testimage.bitwise_and(cloudshadowbit)).eq(0)
    cloudmask = bqa.bitwise_and(cloudbit+cloudshadowbit).eq(0)
    cloudmask = cloudmask.rename(['cloudMask'])
    image = image.addBands(cloudmask)
    image.updateMask('cloudMask')
    return image

In [191]:
b1 = b0.map(cld_mask)
Map = add_composites_to_map(b1,Map,'Cloud Masking')

    - b2: with clouds and shadow masked
    

In [194]:
def testmask():
    m = geemap.Map(center = house_coords,zoom=8)
    testimage = ls.filter(ee.Filter.date(0,'2003-05-31')).filterMetadata('CLOUD_COVER','greater_than',50).sort('CLOUD_COVER').first()
    cld_maskimage = cld_mask(testimage).select('cloudMask')
    shad_maskimage = cld_shdw_mask(testimage).select('cloudMask')
    m.add_ee_layer(testimage,shown=False,name='raw_image',vis_params={'bands':['B3','B2','B1'],'min':0,'max':0.4})
    m.add_ee_layer(shad_maskimage,vis_params={'min':0,'max':1},name='cloud and shadow mask')
    m.add_ee_layer(cld_maskimage,vis_params={'min':0,'max':1},name='cloud mask')
    return m

In [195]:
b2 = b0.map(cld_shdw_mask)

Map = add_composites_to_map(b2,Map,'Cloud and Shadow masking')
Map

Map(bottom=101416.0, center=[38, -76.21], controls=(WidgetControl(options=['position', 'transparent_bg'], widg…

In [196]:
m = geemap.Map(center = house_coords,zoom=10)
vp =  {'bands':['cloudMask'],
              'min':0,
             'max':1,
            'gamma': 1.2,
              }
#m.add_ee_layer(b0.mean(),vis_params=vp)
m.add_ee_layer(b0.map(cld_mask).mean(),vis_params=vp)
m.add_ee_layer(b0.map(cld_shdw_mask).mean(),vis_params=vp)
m

Map(center=[38, -76.21], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

- C: For every step of A&B visualize different reducers for the RGB-bands: 
    - mean 
    - median 
    - count
        - 0.05 percentile
        - 0.95 percentile


- Visualize the results as a true color (for the mean, median and percentiles) and pseudo-color image for a pre- and post-SLC failure year (More background on the SLC-failure in https://www.usgs.gov/core-science-systems/nli/landsat/landsat-7?qt-science_support_page_related_con=0#qt-science_support_page_related_con )


In [198]:
pre_failure_coll = ls.filter(ee.Filter.date(0,'2003-05-31'))
post_failure_coll = ls.filterDate('2003-05-31','2021-09-20')


In [199]:
slc_map = geemap.Map(center=house_coords,zoom=10)
slc_map = add_composites_to_map(pre_failure_coll,slc_map,'Pre failure')
slc_map = add_composites_to_map(post_failure_coll,slc_map,'Post Failure')
slc_map.add_ee_layer(pre_failure_coll.median(),vis_params={'bands':['B4','B3','B2'],'max':0,'min':0.4,'gamma': [0.95, 1.1, 1]})

In [200]:
slc_map

Map(center=[38, -76.21], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…


- **Q1: What are the differences you observe between the different composites (e.g. number of scenes, cloud/noise/SLC artefacts, reflectance values and visual appearance) and explain their cause?**
- **Q2: Discuss what the impact of these differences could be on subsequent big data analysis and how it might affect the veracity of the analysis.**

## Step 3 Greenest pixel composites
- Use the map-principle to calculate the normalized difference vegetation index (NDVI) for every image and use this index to reduce the collection to a greenest pixel composite for every year.
- Visualize the results and compare them to the results of Step 2.
- **Q3: What are the differences between the greenest pixel composite and the results of step 2 and explain their cause?**
- **Q4: What are the advantages and potential disadvantages of using a quality mosaic?**

## Step 4 SLC- and cloud masking
- Load the cloud+cloud-shadow mask for every image by extracting the BQA mask.
- Apply different image dilation/ erosion / opening / closing operations on the BQA mask with different distances/iterations.
- Update the mask with the dilated/ erosed / opened / closed BQA mask.
- Pick one image and visualize the effect of the dilation/ erosion / opening / closing on the data.
- **Q5: Visualize and explain the differences of the different dilation/erosion/ opening/closing operations on the mask. What would be the different dis/advantages of the different methods?**

## Step 5 SLC-filling
- Apply a map-reduce function to the USGS Landsat 7 Top of Atmosphere Reflectance Tier 1 since 2003 where you replace the missing SLC/cloud/shadow data of every image by:
    - The median reflectance
    - The mean reflectance
    - The reflectance of the greenest pixel
    - The reflectance of the pixel that is closest in time
- Visualize two images (summer vs winter) where you illustrate the effect of the different filling scenarios.
- **Q6: Visualize and explain the differences of the different filling scenarios. What would be the different dis/advantages of the different methods?**